# Redes Neurais Convolucionais

### Descriçao

---
1. O objetivo deste notebook é Redes Neurais Convolucionais.
2. Os dados para usar serão clonados do nosso próprio github, pela pasta dataset.
3. Vamos treinar nosso modelo para que ele aprenda a detectar doença em plantações de tomate.
---


### Dicionário


Class	                                                  | Type  	  |    Description                              |
----------------------------------------------------------|:---------:|:-------------------------------------------:|
tomato_mosaic_virus  	  										  	  |Class     | Imagem dos mosaico de vírus	                    |
Target Spot														  |Class    | Ponto alvo                         |
Bacterial_spot 		     										  |Class     | Pontos com bactéria	                |
Tomato_Yellow_Leaf_Curl_Virus  | Class | Vírus do enrolamento da folha amarela do tomate | float | nota final do ensino médio
Late_blight  | Class | Folhas queimadas
Leaf_Mold | Class | Molde de folha
Early_blight | Class | Preconceito
Spider_mites Two-spotted_spider_mite  | Class | Ácaros e outras pragas
Tomato___healthy   | Class | Saúde dos Tomates
Septoria_leaf_spot   | Class | Mancha foliar de Septoria


# Documentação

1. ** Pandas ** -> [Link](https://pandas.pydata.org/docs/)
2. ** Numpy ** -> [Link](https://numpy.org/doc/)
4. ** Scikit Learn ** -> [Link](https://scikit-learn.org/stable/)
5. ** Keras ** -> [Link](https://keras.io/api/)
6. ** Tensor Flow ** -> [Tensor Flow](https://www.tensorflow.org/api_docs/python/tf/keras)

# Instalação de pacotes

In [ ]:
!pip install torch torchvision torchaudio pandas numpy keras

# Obtendo o dataset

In [ ]:
!git clone https://github.com/batestin1/coding_the_future_dio_redes_neurais.git #clona o repositorio
!mv coding_the_future_dio_redes_neurais/dataset /content/ #move apenas a pasta dataset para fora do diretorio
!rm -rf coding_the_future_dio_redes_neurais #exclui o restante que nao nos interessa



# Bibliotecas

In [ ]:
import numpy as np
import os

from keras.models import Sequential
from keras.layers import Conv2D #o metodo que consegue ler imagem e transformar em bits
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.preprocessing.image import ImageDataGenerator # Para o pre-processamento de imagens
from tensorflow.keras.utils import plot_model
from keras.models import load_model #para salvar modelos do keras

from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor


# Construindo a RNC

In [ ]:
# Invocando o metódo
rnc = Sequential()

In [ ]:
#Criando Camadas

rnc.add(Conv2D(32, (3, 3), input_shape=(128, 128, 3), activation='relu'))
rnc.add(MaxPooling2D(pool_size=(2, 2)))

rnc.add(Conv2D(16, (3, 3), activation='relu'))
rnc.add(MaxPooling2D(pool_size=(2, 2)))

rnc.add(Conv2D(8, (3, 3), activation='relu'))
rnc.add(MaxPooling2D(pool_size=(2, 2)))

In [ ]:
# Flattening
rnc.add(Flatten())

In [ ]:
# Dense layers
rnc.add(Dense(units=128, activation='relu'))
rnc.add(Dense(units=1, activation='sigmoid'))


In [ ]:
# Copilando a rede
rnc.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

#rnc.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
rnc.summary()

classifier

# Pré-processamento das imagens


#### Treino

In [ ]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
training_set = train_datagen.flow_from_directory(
        '/content/dataset/rnc/train',
        target_size=(128, 128),
        batch_size=64,
        class_mode='categorical' )

In [ ]:
label_map = (training_set.class_indices)

In [ ]:
label_map

#### Teste

In [ ]:
test_set = test_datagen.flow_from_directory(
        '/content/dataset/rnc/test',
        target_size=(128, 128),
        batch_size=64,
        class_mode='categorical')

 # Criando o Modelo

In [ ]:
rnc.fit_generator(
        training_set,
        steps_per_epoch=20,
        epochs=10000,
        validation_data=test_set,
        validation_steps=100)



# Salvando o modelo

In [ ]:
folder = 'rnc/'

# Verifica se o diretório existe e, se não existir, cria o diretório
if not os.path.exists(folder):
    os.makedirs(folder)

# Salva o modelo no diretório especificado
rnc.save_weights(os.path.join(folder, 'rede_neural_convolucional.h5'))

# Bonus Track

### Você conhece o [Teachable Machine](https://teachablemachine.withgoogle.com/train)?

1. Plataforma low code de treinamento de redes convolucionais
2. exporta em modelos .h5
3. Setup simples e intuitivo


# Importando os modelos

In [ ]:

model = load_model(os.path.join(folder, 'rede_neural_convolucional.h5'))